In [25]:
import os
import json
import shutil
from tqdm.notebook import tqdm

label_dir = './annotations/'
image_dir = './images/'
to_label_dir = './yolo/labels/'
to_image_dir = './yolo/images/'

label_paths = [os.path.join(label_dir, i).replace('\\', '/') for i in os.listdir(label_dir)]

In [ ]:
n = 0
for label_path in label_paths:
    
    mode = label_path.split('_')[-1].split('.')[0]
    label_data = json.load(open(label_path))

    label_data['images']
    image_dict = {}
    for i in label_data['images']:
        image_dict[i['id']] = [i['width'], i['height'], i['file_name']]

    label_dict = {}
    for k in image_dict.keys():
        label_dict[k] = []
    for label in label_data['annotations']:
        x1, y1, w, h = label['bbox']

        image_id = label['image_id']
        W, H, _ = image_dict[image_id]
        x_c = (x1 + w/2) / W
        y_c = (y1 + h/2) / H
        _w = w / W
        _h = h / H

        label_dict[image_id].append('0 {} {} {} {}'.format(x_c, y_c, _w, _h))

    for k, v in tqdm(label_dict.items()):
        if len(v) > 0:
            _, _, file_name = image_dict[k]
#             file_name = n
            image_path = os.path.join(image_dir, file_name).replace('\\', '/')
            to_image_path = os.path.join(to_image_dir, mode, '{}.jpg'.format(n)).replace('\\', '/')
            to_label_path = os.path.join(to_label_dir, mode, '{}.txt'.format(n)).replace('\\', '/')

            if os.path.exists(to_label_path):
                print(to_label_path)
            with open(to_label_path, 'w', encoding='utf-8') as f:
                for i in v:
                    f.write(i)
                    f.write('\n')
            shutil.copyfile(image_path, to_image_path)
            n += 1

In [67]:
import json
from tqdm.notebook import tqdm

In [104]:
# data_paths = ['./annotations/tablebank_word_val.json', './annotations/tablebank_latex_val.json', './annotations/tablebank_word_test.json', './annotations/tablebank_latex_test.json']
data_paths = ['./annotations/tablebank_word_train.json', './annotations/tablebank_latex_train.json']
save_path = './annotations/tablebank_all_train.json'

# 重建images信息列表
image_msgs = {}
categories = {}
annotation_msgs = []

for data_path in data_paths:
    with open(data_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    id2filename = {}
    id2labelname = {}

    for image_msg in tqdm(data['images']):
        file_name = image_msg['file_name']
        id = image_msg['id']

        image_msgs[file_name] = image_msg
        id2filename[id] = file_name
    
    for category in tqdm(data['categories']):
        label_name = category['name']
        id = category['id']

        categories[label_name] = category
        id2labelname[id] = label_name

    # annotations里面的category_id, image_id, 改为名称信息
    for annotation in tqdm(data['annotations']):
        category_id = annotation['category_id']
        image_id = annotation['image_id']

        file_name = id2filename[image_id]
        label_name = id2labelname[category_id]

        annotation['file_name'] = file_name
        annotation['label_name'] = label_name

        annotation_msgs.append(annotation)

# 重置image_msg里面的id信息
for n, (k, v) in enumerate(image_msgs.items()):
    v['id'] = n + 1

# 重置categories 里面的id信息
for n, (k, v) in enumerate(categories.items()):
    v['id'] = n + 1

for n, d in enumerate(annotation_msgs):
    d['id'] = n + 1

# 更新 image_id, category_id
for d in annotation_msgs:
    file_name = d['file_name']
    label_name = d['label_name']

    d['category_id'] = categories[label_name]['id']
    d['image_id'] = image_msgs[file_name]['id']

    d.pop('file_name')
    d.pop('label_name')

# 合并到初始的json文件里面
with open(data_paths[0], 'r', encoding='utf-8') as f:
    main_data = json.load(f)

main_data['annotations'] = annotation_msgs
main_data['images'] = [i for i in image_msgs.values()]
main_data['categories'] = [i for i in categories.values()]

with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(main_data, f)

  0%|          | 0/73383 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/95404 [00:00<?, ?it/s]

  0%|          | 0/187199 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/237431 [00:00<?, ?it/s]

In [98]:
main_data['annotations']

[{'category_id': 1,
  'area': 46172,
  'iscrowd': 0,
  'segmentation': [[89, 316, 89, 435, 477, 435, 477, 316]],
  'id': 1,
  'image_id': 22606,
  'bbox': [89, 316, 388, 119]},
 {'category_id': 1,
  'area': 27393,
  'iscrowd': 0,
  'segmentation': [[89, 423, 89, 492, 486, 492, 486, 423]],
  'id': 2,
  'image_id': 32907,
  'bbox': [89, 423, 397, 69]},
 {'category_id': 1,
  'area': 45600,
  'iscrowd': 0,
  'segmentation': [[84, 87, 84, 201, 484, 201, 484, 87]],
  'id': 3,
  'image_id': 57515,
  'bbox': [84, 87, 400, 114]},
 {'category_id': 1,
  'area': 55188,
  'iscrowd': 0,
  'segmentation': [[84, 554, 84, 700, 462, 700, 462, 554]],
  'id': 4,
  'image_id': 11764,
  'bbox': [84, 554, 378, 146]},
 {'category_id': 1,
  'area': 95887,
  'iscrowd': 0,
  'segmentation': [[89, 501, 89, 754, 468, 754, 468, 501]],
  'id': 5,
  'image_id': 43643,
  'bbox': [89, 501, 379, 253]},
 {'category_id': 1,
  'area': 96695,
  'iscrowd': 0,
  'segmentation': [[84, 531, 84, 764, 499, 764, 499, 531]],
  'id'